In [51]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\wziller\Milwaukee Tool\_Global AME\MSOE\CS5610\M6\csc5610base\GroupProjectM6\Awards_Batting_HoF.csv'
fe_batting = pd.read_csv(file_path)

# Perform feature engineering


# Group by playerID and aggregate relevant statistics
grouped_df = fe_batting.groupby('playerID').agg(
    years_played=('Batting_df_yearID', lambda x: x.nunique()),
    total_games=('Batting_df_G', 'sum'),
    total_at_bats=('Batting_df_AB', 'sum'),
    total_runs=('Batting_df_R', 'sum'),
    total_hits=('Batting_df_H', 'sum'),
    total_walks=('Batting_df_BB', 'sum'),
    total_doubles=('Batting_df_2B', 'sum'),
    total_triples=('Batting_df_3B', 'sum'),
    total_home_runs=('Batting_df_HR', 'sum'),
    total_RBI=('Batting_df_RBI', 'sum'),
    total_SB=('Batting_df_SB', 'sum'),
    total_CS=('Batting_df_CS', 'sum'),
    total_BB=('Batting_df_BB', 'sum'),
    total_SO=('Batting_df_SO', 'sum'),
    total_IBB=('Batting_df_IBB', 'sum'),
    total_HBP=('Batting_df_HBP', 'sum'),
    total_SH=('Batting_df_SH', 'sum'),
    total_SF=('Batting_df_SF', 'sum'),
    total_GIDP=('Batting_df_GIDP', 'sum'),
    max_HR=('Batting_df_HR', 'max'),
    max_hits=('Batting_df_H', 'max'),
    max_SB=('Batting_df_SB', 'max'),
    HOF_status=('HallOfFame_df_inducted', 'max')
    # HOF_year=('HallOfFame_df_yearid', 'max'),
    # BattingPost_G=('BattingPost_df_G', 'sum'),
    # BattingPost_AB=('BattingPost_df_AB', 'sum'),
    # BattingPost_R=('BattingPost_df_R', 'sum'),
    # BattingPost_H=('BattingPost_df_H', 'sum'),
    # BattingPost_2B=('BattingPost_df_2B', 'sum'),
    # BattingPost_3B=('BattingPost_df_3B', 'sum'),
    # BattingPost_HR=('BattingPost_df_HR', 'sum'),
    # BattingPost_RBI=('BattingPost_df_RBI', 'sum'),
    # BattingPost_SB=('BattingPost_df_SB', 'sum'),
    # BattingPost_CS=('BattingPost_df_CS', 'sum'),
    # BattingPost_BB=('BattingPost_df_BB', 'sum'),
    # BattingPost_SO=('BattingPost_df_SO', 'sum'),
    # BattingPost_IBB=('BattingPost_df_IBB', 'sum'),
    # BattingPost_HBP=('BattingPost_df_HBP', 'sum'),
    # BattingPost_SH=('BattingPost_df_SH', 'sum'),
    # BattingPost_SF=('BattingPost_df_SF', 'sum'),
    # BattingPost_GIDP=('BattingPost_df_GIDP', 'sum')
).reset_index()

grouped_df['total_singles'] = grouped_df['total_hits'] - (grouped_df['total_doubles'] + grouped_df['total_triples'] + grouped_df['total_home_runs'])

# # Merge back with the original DataFrame to preserve all columns
# merge_df = pd.merge(grouped_df, grouped_df, on='playerID', how='left')

# Calculate new features
grouped_df['batting_avg'] = grouped_df['total_hits'] / grouped_df['total_at_bats']
grouped_df['OBP'] = (grouped_df['total_hits'] + grouped_df['total_walks']) / (grouped_df['total_at_bats'] + grouped_df['total_walks'])
grouped_df['SLG'] = (
    (grouped_df['total_singles'] - grouped_df['total_doubles'] - grouped_df['total_triples'] - grouped_df['total_home_runs']) +
    (2 * grouped_df['total_doubles']) +
    (3 * grouped_df['total_triples']) +
    (4 * grouped_df['total_home_runs'])
) / grouped_df['total_at_bats']
grouped_df['OPS'] = grouped_df['OBP'] + grouped_df['SLG']

#27 

# Replace empty values in engineered features with 0
engineered_features = [
    'years_played', 'total_games', 'total_at_bats', 'total_runs', 'total_hits',
    'total_walks', 'total_doubles', 'total_triples', 'total_home_runs',
    'total_RBI', 'total_SB', 'total_CS', 'total_BB', 'total_SO', 'total_IBB',
    'total_HBP', 'total_SH', 'total_SF', 'total_GIDP', 'max_HR', 'max_hits',
    'max_SB', 'total_singles', 'batting_avg', 'OBP', 'SLG', 'OPS'
]

# Fill NaN values with 0
grouped_df[engineered_features] = grouped_df[engineered_features].fillna(0)

# Row count before filtering
print(f"Original dataset size: {len(grouped_df)}")

# Apply minimum thresholds for filtering
min_games = 50
min_at_bats = 100
min_ops = 0.01

filtered_df = grouped_df[
    (grouped_df['total_games'] >= min_games) &
    (grouped_df['total_at_bats'] >= min_at_bats) &
    (grouped_df['OPS'] >= min_ops)
]

# Row count after filtering
print(f"Filtered dataset size: {len(filtered_df)}")


# Convert data types
filtered_df['HOF_status'] = filtered_df['HOF_status'].astype('category')
filtered_df['playerID'] = filtered_df['playerID'].astype('string')



# # Identify HOF players in the original dataset
# original_hof_players = grouped_df[grouped_df['HOF_status'] == 1]['playerID']

# # Identify HOF players in the filtered dataset
# filtered_hof_players = filtered_df[filtered_df['HOF_status'] == 1]['playerID']

# # Compare and identify missing HOF players
# hof_players_filtered_out = original_hof_players[~original_hof_players.isin(filtered_hof_players)]

# # Display the result
# if hof_players_filtered_out.empty:
#     print("No Hall of Fame players were filtered out.")
# else:
#     print("The following Hall of Fame players were filtered out:")
#     print(hof_players_filtered_out)


# Save filtered DataFrame
filtered_df.to_csv('Award_Batting_HOF_FE_csv.csv', index=False)
filtered_df.to_feather('Award_Batting_HOF_FE.feather')

filtered_df.info()


# # Save filtered DataFrame
# grouped_df.to_csv('Award_Batting_HOF_FE_csv.csv', index=False)
# grouped_df.to_feather('Award_Batting_HOF_FE.feather')

# grouped_df.info()

Original dataset size: 18724
Filtered dataset size: 8325
<class 'pandas.core.frame.DataFrame'>
Index: 8325 entries, 1 to 18722
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   playerID         8325 non-null   string  
 1   years_played     8325 non-null   int64   
 2   total_games      8325 non-null   float64 
 3   total_at_bats    8325 non-null   float64 
 4   total_runs       8325 non-null   float64 
 5   total_hits       8325 non-null   float64 
 6   total_walks      8325 non-null   float64 
 7   total_doubles    8325 non-null   float64 
 8   total_triples    8325 non-null   float64 
 9   total_home_runs  8325 non-null   float64 
 10  total_RBI        8325 non-null   float64 
 11  total_SB         8325 non-null   float64 
 12  total_CS         8325 non-null   float64 
 13  total_BB         8325 non-null   float64 
 14  total_SO         8325 non-null   float64 
 15  total_IBB        8325 non-null   flo

C:\Users\wziller\AppData\Local\Temp\ipykernel_58676\309116741.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['HOF_status'] = filtered_df['HOF_status'].astype('category')
C:\Users\wziller\AppData\Local\Temp\ipykernel_58676\309116741.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['playerID'] = filtered_df['playerID'].astype('string')


In [52]:
# grouped_df.head()
filtered_df.head()


,playerID,years_played,total_games,total_at_bats,total_runs,total_hits,total_walks,total_doubles,total_triples,total_home_runs,...,total_GIDP,max_HR,max_hits,max_SB,HOF_status,total_singles,batting_avg,OBP,SLG,OPS
1,aaronha01,23,3298.0,12364.0,2174.0,3771.0,1402.0,624.0,98.0,755.0,...,328.0,47.0,223.0,31.0,True,2294.0,0.304998,0.375781,0.435053,0.810834
2,aaronto01,7,437.0,944.0,102.0,216.0,86.0,42.0,6.0,13.0,...,36.0,8.0,77.0,6.0,False,155.0,0.228814,0.293204,0.262712,0.555916
7,abbated01,9,855.0,3044.0,355.0,772.0,289.0,99.0,43.0,11.0,...,0.0,3.0,170.0,35.0,False,619.0,0.253614,0.318332,0.274967,0.593299
8,abbeybe01,5,79.0,225.0,21.0,38.0,21.0,3.0,3.0,0.0,...,0.0,0.0,12.0,2.0,False,32.0,0.168889,0.239837,0.182222,0.422060
9,abbeych01,5,451.0,1751.0,307.0,492.0,167.0,67.0,46.0,19.0,...,0.0,8.0,164.0,31.0,False,360.0,0.280982,0.343587,0.328955,0.672542


In [53]:
# grouped_df.describe()
filtered_df.describe()

,years_played,total_games,total_at_bats,total_runs,total_hits,total_walks,total_doubles,total_triples,total_home_runs,total_RBI,...,total_SF,total_GIDP,max_HR,max_hits,max_SB,total_singles,batting_avg,OBP,SLG,OPS
count,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,...,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000,8325.000000
mean,8.048769,588.736697,1818.207447,245.279399,482.053333,169.337778,81.551111,17.323844,36.266186,221.232312,...,8.468468,27.800721,7.278318,84.492853,9.025105,346.912192,0.230680,0.285455,0.265181,0.550636
std,4.762931,672.442511,2500.667029,398.246365,735.073031,284.801444,129.743515,35.437180,82.181671,371.862152,...,18.212554,50.794231,9.748693,62.304756,13.917161,520.325407,0.050712,0.061143,0.068502,0.126451
min,1.000000,50.000000,100.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.029126,0.059322,0.029126,0.094547
25%,4.000000,151.000000,291.000000,27.000000,59.000000,18.000000,9.000000,1.000000,1.000000,23.000000,...,0.000000,0.000000,1.000000,25.000000,1.000000,46.000000,0.205674,0.253309,0.226721,0.483549
50%,7.000000,340.000000,774.000000,86.000000,180.000000,55.000000,28.000000,5.000000,6.000000,74.000000,...,0.000000,6.000000,3.000000,73.000000,4.000000,135.000000,0.242171,0.298625,0.275117,0.576899
75%,11.000000,791.000000,2399.000000,301.000000,616.000000,203.000000,103.000000,19.000000,31.000000,263.000000,...,8.000000,32.000000,10.000000,138.000000,11.000000,447.000000,0.265306,0.327831,0.312281,0.638568
max,27.000000,5700.000000,21752.000000,3992.000000,7028.000000,3416.000000,1584.000000,618.000000,1146.000000,4152.000000,...,236.000000,630.000000,73.000000,262.000000,138.000000,5286.000000,0.366363,0.480621,0.528340,1.000137
